# Preparation

In [ ]:
! pip install aim datasets==2.19.0 nltk==3.8.1 numpy==1.26.4 quickumls==1.4.2 scikit-learn==1.4.2 seqeval==1.2.2 spacy==3.3.0 torch==2.2.1 tqdm==4.65.0 transformers==4.39.3

  Using cached pydantic-1.8.2-py3-none-any.whl.metadata (103 kB)
Using cached pydantic-1.8.2-py3-none-any.whl (126 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.7
    Uninstalling pydantic-1.10.7:
      Successfully uninstalled pydantic-1.10.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires pydantic>=2.9.2, but you have pydantic 1.8.2 which is incompatible.
langchain-core 0.3.45 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.8.2 which is incompatible.
langchain 0.3.20 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.8.2 which is incompatible.
openai 1.66.3 requires pydantic<3,>=1.9.0, but you have pydantic 1.8.2 which is incompatible.
wandb 0.19.8 requires pydantic<3,>=2.6, but you have pydantic 1.8.2 which is incompatible.
google-genai 1.5.0 re

In [ ]:
# resolve compatilty of aim
!pip install pydantic==1.10.7

  Using cached pydantic-1.10.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (145 kB)
Using cached pydantic-1.10.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.17 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.10.7 which is incompatible.
spacy 3.3.0 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.10.7 which is incompatible.
albumentations 2.0.5 requires pydantic>=2.9.2, but you have pydantic 1.10.7 which is incompatible.
langchain-core 0.3.45 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.7 which is incompatible.

In [ ]:
!pip uninstall pandas
!pip install pandas==2.1.4

Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/pandas-2.1.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pandas/*
Proceed (Y/n)? n


In [ ]:
import pandas as pd

In [ ]:
pd.__version__

'2.1.4'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Get Data
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

# MedDec dataset
mimic_path = '/content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/05 Generated File/meddec-mimic-iii'


# Phenotype Annotations
annotations_path = '/content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/05 Generated File/pheno_annoations_ACTdb102003.csv'


notes_path='/content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/03 Data/NOTEEVENTS.csv'

Mounted at /content/drive


In [ ]:
from glob import glob
import pandas as pd
import os, sys


if len(sys.argv) != 3:
    print('Usage: python extract_texts.py <data_dir> <notes_path>')
    sys.exit(1)

data_dir = mimic_path
text_dir = os.path.join(data_dir, 'raw_text')
files = glob(os.path.join(data_dir, 'data/*.json')) #'data/*.json'


# Extract Notes Text from MIMIC-III (No need to run)

- Attention, the generated extracted notes are stocked in the folder: Generated file to avoid conflicts.  /content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/05 Generated File/meddec-mimic-iii

# preprocess phenos (No need to Run)
- (transforme none to '? ')

In [ ]:
import pandas as pd
import os, sys

# Constants
PHENOTYPE_COLUMNS = ['ADVANCED.CANCER', 'ADVANCED.HEART.DISEASE', 'ADVANCED.LUNG.DISEASE', 'ALCOHOL.ABUSE',
                     'CHRONIC.NEUROLOGICAL.DYSTROPHIES', 'CHRONIC.PAIN.FIBROMYALGIA', 'DEPRESSION', 'OBESITY',
                     'OTHER.SUBSTANCE.ABUSE', 'PSYCHIATRIC.DISORDERS', 'NONE']

In [ ]:
# Aggregate the annotations of the same SUBJECT_ID, HADM_ID, ROW_ID into a single phenotype label.
# If there is only one annotation, use that as the gold data.
# If there are multiple annotations, prioritize annotators based on the following order:
# DAG or PAT are prioritized over all other annotators.
# JF or JTW are prioritized over ETM and JW.
# If there are multiple annotations from the same prioritized annotator, use the sum of the annotations.
# If one of the annotations is NONE or UNSURE, the phenotype label is '?'.

def aggregate_annotations(df):
    def process_group(group):
        if len(group) == 1:
            return process_single_annotation(group.iloc[0])
        else:
            return process_multiple_annotations(group)

    def process_single_annotation(row):
        phenotype_label = [col if col != 'UNSURE' else '?' for col in PHENOTYPE_COLUMNS if row[col] > 0]
        return pd.Series({'phenotype_label': ','.join(phenotype_label) if phenotype_label else '?'})

    def process_multiple_annotations(group):
        priority_operators = [['DAG', 'PAT'], ['JTW', 'JF'], ['ETM', 'JW']]
        for operator_group in priority_operators:
            if group['OPERATOR'].isin(operator_group).any():
                selected_rows = group[group['OPERATOR'].isin(operator_group)]
                break
        else:
            selected_rows = group

        selected_rows_unique = selected_rows.drop(['BATCH.ID', 'OPERATOR'], axis=1).drop_duplicates()

        if len(selected_rows_unique) > 1 and selected_rows_unique['NONE'].sum() > 0:
            return pd.Series({
                'phenotype_label': '?',
                'OPERATOR': ','.join(selected_rows['OPERATOR'].unique())
            })

        # Sum over phenotype_columns and keep other unchanged
        selected_rows_unique = selected_rows_unique.sum()
        selected_rows_unique['OPERATOR'] = ','.join(selected_rows['OPERATOR'].unique())

        # selected_rows = selected_rows_unique.sum()
        phenotype_label = [col if col != 'UNSURE' else '?' for col in PHENOTYPE_COLUMNS if selected_rows_unique[col] > 0]
        return pd.Series({'phenotype_label': ','.join(sorted(phenotype_label)) if phenotype_label else '?',
                'OPERATOR': ",".join(selected_rows['OPERATOR'].unique())})

    return df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(process_group).reset_index()

In [ ]:
# Paths
INPUT_FILE = annotations_path
OUTPUT_FILE = os.path.join(os.path.dirname(INPUT_FILE), 'phenos.csv')

# Read and preprocess data
df = pd.read_csv(INPUT_FILE)
df['PSYCHIATRIC.DISORDERS'] = df['DEMENTIA'] | df['DEVELOPMENTAL.DELAY.RETARDATION'] | df['SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS']
df = df[['SUBJECT_ID', 'HADM_ID', 'ROW_ID'] + PHENOTYPE_COLUMNS + ['OPERATOR', 'BATCH.ID']]

# Aggregate annotations
result_df = aggregate_annotations(df)

In [ ]:
result_df

,SUBJECT_ID,HADM_ID,phenotype_label,OPERATOR
0,109,164029,NONE,JTW
1,154,102354,"DEPRESSION,PSYCHIATRIC.DISORDERS",JTW
2,154,162891,NONE,JTW
3,188,160697,?,"JF,JTW"
4,188,191517,NONE,JTW
...,...,...,...,...
808,81444,100960,"ADVANCED.LUNG.DISEASE,PSYCHIATRIC.DISORDERS",JTW
809,85999,134661,"ADVANCED.LUNG.DISEASE,CHRONIC.PAIN.FIBROMYALGI...",JTW
810,88206,155516,NONE,JTW
811,94525,154715,"ALCOHOL.ABUSE,DEPRESSION,OTHER.SUBSTANCE.ABUSE",JTW


In [ ]:
phenos=pd

,HADM_ID,SUBJECT_ID,ROW_ID,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,BATCH.ID,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEMENTIA,DEPRESSION,DEVELOPMENTAL.DELAY.RETARDATION,NON.ADHERENCE,NONE,OBESITY,OPERATOR,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS,UNSURE
0,100103,3365,17170,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,JTW,0,0,0
1,100103,3365,17170,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,ETM,0,0,0
2,100137,27290,50828,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,JF,0,0,0
3,100137,27290,50828,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,JW,0,0,0
4,100473,5525,52670,0,0,0,0,18Feb16,0,0,0,0,0,0,1,0,JW,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,199807,23549,2787,0,1,1,0,07Dec15,0,0,0,0,0,0,0,0,ETM,0,0,0
2272,199883,7029,18731,0,0,0,1,18Feb16,0,0,0,0,0,1,0,0,JW,0,0,0
2273,199883,7029,18731,0,0,0,1,18Feb16,0,0,0,0,0,1,0,0,JF,0,0,0
2274,199884,1931,8180,0,1,0,0,16Mar16,0,0,0,1,0,0,0,0,DAG,0,0,0


# change file name for better match (No Touch)

In [ ]:
target_path1=os.path.join(data_dir,'raw_text')

In [ ]:
import os

# 设置文件夹路径（Google Drive 或 Colab 本地路径）
folder_path = target_path1

# 查找所有以 `.txt` 结尾的文件
files = glob(os.path.join(folder_path, "*.txt"))

# 遍历文件并重命名
for file in files:
    filename = os.path.basename(file)  # 获取文件名
    parts = filename.split("_")  # 按 "_" 拆分

    if len(parts) >= 2:  # 确保有足够的部分
        new_filename = f"{parts[0]}_{parts[1]}.txt"  # 仅保留前两部分
        new_path = os.path.join(folder_path, new_filename)

        os.rename(file, new_path)  # 重命名文件
        print(f"Renamed: {filename} → {new_filename}")

print("✅ 文件重命名完成！")


Renamed: 11216_188044_9756.txt → 11216_188044.txt
Renamed: 10814_119849_52793.txt → 10814_119849.txt
Renamed: 10814_141014_52776.txt → 10814_141014.txt
Renamed: 11063_166859_11978.txt → 11063_166859.txt
Renamed: 11043_138702_1715.txt → 11043_138702.txt
Renamed: 10814_171527_52779.txt → 10814_171527.txt
Renamed: 10814_180749_52780.txt → 10814_180749.txt
Renamed: 10814_101543_52781.txt → 10814_101543.txt
Renamed: 10997_105782_54153.txt → 10997_105782.txt
Renamed: 11043_165605_1717.txt → 11043_165605.txt
Renamed: 11043_171810_1714.txt → 11043_171810.txt
Renamed: 11398_109233_25673.txt → 11398_109233.txt
Renamed: 11236_179800_52417.txt → 11236_179800.txt
Renamed: 11559_103284_21540.txt → 11559_103284.txt
Renamed: 11236_167863_52420.txt → 11236_167863.txt
Renamed: 11235_145044_903.txt → 11235_145044.txt
Renamed: 11235_147720_904.txt → 11235_147720.txt
Renamed: 11678_145492_25150.txt → 11678_145492.txt
Renamed: 11235_158824_902.txt → 11235_158824.txt
Renamed: 11645_134417_25627.txt → 11645_1

In [ ]:
target_path2=os.path.join(data_dir,'data')

In [ ]:
import os

# 设置文件夹路径（Google Drive 或 Colab 本地路径）
folder_path = target_path2

# 查找所有以 `.txt` 结尾的文件
files = glob(os.path.join(folder_path, "*.json"))
# 遍历文件并重命名
for file in files:
    filename = os.path.basename(file)  # 获取文件名
    parts = filename.split("_")  # 按 "_" 拆分

    if len(parts) >= 2:  # 确保有足够的部分
        new_filename = f"{parts[0]}_{parts[1]}.json"  # 仅保留前两部分
        new_path = os.path.join(folder_path, new_filename)

        os.rename(file, new_path)  # 重命名文件
        print(f"Renamed: {filename} → {new_filename}")

print("✅ 文件重命名完成！")

Renamed: 11216_188044_9756.json → 11216_188044.json
Renamed: 10814_119849_52793.json → 10814_119849.json
Renamed: 11043_171810_1714.json → 11043_171810.json
Renamed: 10814_180749_52780.json → 10814_180749.json
Renamed: 11043_138702_1715.json → 11043_138702.json
Renamed: 10814_171527_52779.json → 10814_171527.json
Renamed: 10814_141014_52776.json → 10814_141014.json
Renamed: 11063_166859_11978.json → 11063_166859.json
Renamed: 11043_165605_1717.json → 11043_165605.json
Renamed: 10997_105782_54153.json → 10997_105782.json
Renamed: 10814_101543_52781.json → 10814_101543.json
Renamed: 11398_109233_25673.json → 11398_109233.json
Renamed: 11236_179800_52417.json → 11236_179800.json
Renamed: 11559_103284_21540.json → 11559_103284.json
Renamed: 11235_145044_903.json → 11235_145044.json
Renamed: 11236_167863_52420.json → 11236_167863.json
Renamed: 11235_147720_904.json → 11235_147720.json
Renamed: 11236_185878_52418.json → 11236_185878.json
Renamed: 11235_158824_902.json → 11235_158824.json
Ren

## no need to re execute

In [ ]:
pd.__version__

'2.1.4'

In [ ]:
notes = pd.read_csv(notes_path, nrows=100000, low_memory=False, dtype=str, converters={'ROW_ID': int})

In [ ]:
os.makedirs(text_dir, exist_ok=True)
for fn in files:
    sid, hadm, rid = map(int, os.path.splitext(os.path.basename(fn))[0].split('_'))
    if (sid, hadm, rid) not in notes.index:
        continue
    note = notes.loc[sid, hadm, rid]
    out_fn = f'{sid}_{hadm}_{rid}.txt'
    with open(os.path.join(text_dir, out_fn), 'w') as f:
        f.write(note.TEXT)

In [ ]:
mimic_path

'/content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/05 Generated File/meddec-mimic-iii'

# Main Execution

In [ ]:
#only important when you execute on colab, locally, just make sure the packages.py are placed in the same folder as this notebook
import sys
sys.path.append('/content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/04 Code')

In [ ]:
import aim

In [ ]:
# import importlib
# import data  # Ensure the script is properly imported
# importlib.reload(data)

<module 'data' from '/content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/04 Code/data.py'>

In [ ]:
import json
import torch
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

from tqdm import tqdm
from data import load_data
from model import load_model

In [ ]:
mimic_path
os.listdir(mimic_path)

['README.md',
 'SHA256SUMS.txt',
 'LICENSE.txt',
 'data',
 'raw_text',
 'pheno_annoations_ACTdb102003.csv',
 'stats.csv',
 'phenos.csv']

## Args

In [ ]:
import argparse
from datetime import datetime
import os

def get_args(notebook_args=None):
    parser = argparse.ArgumentParser()

    # Define all arguments (same as before)
    parser.add_argument('--data_dir', default='./data/')
    parser.add_argument('--ckpt')
    parser.add_argument('--aim_repo', default='.')
    parser.add_argument('--aim_exp', default='mimic-decisions-1215')
    parser.add_argument('--label_encoding', default='multiclass')
    parser.add_argument('--debug', action='store_true')
    parser.add_argument('--save_losses', action='store_true')
    parser.add_argument('--task', default='token', choices=['seq', 'token'])
    parser.add_argument('--max_len', type=int, default=512)
    parser.add_argument('--model', default='roberta-base')
    parser.add_argument('--model_name', default='google/electra-base-discriminator')
    parser.add_argument('--gpu', default='0')
    parser.add_argument('--grad_accumulation', default=2, type=int)
    parser.add_argument('--pheno_id', type=int)
    parser.add_argument('--unseen_pheno', type=int, default=True)
    parser.add_argument('--total_steps', type=int, default=5000)
    parser.add_argument('--train_log', type=int, default=500)
    parser.add_argument('--val_log', type=int, default=1000)
    parser.add_argument('--seed', default='0')
    parser.add_argument('--num_phenos', type=int, default=10)
    parser.add_argument('--num_decs', type=int, default=9)
    parser.add_argument('--batch_size', type=int, default=8)
    parser.add_argument('--pos_weight', type=float, default=1.0)
    parser.add_argument('--truncate_train', action='store_true')
    parser.add_argument('--truncate_eval', action='store_true')
    parser.add_argument('--load_ckpt', action='store_true')
    parser.add_argument('--eval_only', action='store_true')
    parser.add_argument('--lr', type=float, default=4e-5)
    parser.add_argument('--resample', default='')
    parser.add_argument('--verbose', type=bool, default=True)
    parser.add_argument('--use_crf', type=bool)

    # If running in a notebook, override with manually set values
    if notebook_args:
        args = parser.parse_args('')
        for key, value in notebook_args.items():
            setattr(args, key, value)
    else:
        args = parser.parse_args()  # Normal behavior if used in a script

    # Generate checkpoint directory
    curtime = datetime.now().strftime('%m%d_%H-%M-%S')
    args.ckpt_dir = f'./checkpoints/{curtime}-{os.path.basename(args.model_name)}-{args.model}'

    # Convert seed to a list of integers
    args.seed = [int(x) for x in args.seed.split(',')]

    # Define number of labels based on task type
    if args.task == 'seq' and args.pheno_id is not None:
        args.num_labels = 1
    elif args.task == 'seq':
        args.num_labels = args.num_phenos
    elif args.task == 'token':
        args.num_labels = args.num_decs
        if args.label_encoding == 'multiclass':
            args.num_labels = args.num_labels * 2 + 1
        elif args.label_encoding == 'bo':
            args.num_labels *= 2
        elif args.label_encoding == 'boe':
            args.num_labels *= 3

    return args
# Peronalize model parameters

model_params = {
    'data_dir': mimic_path,
    'batch_size': 8,
    'lr': 4e-5,
    'model_name': 'google/electra-base-discriminator',
    'label_encoding': 'multiclass',
    'total_steps': 5000,
    'debug': True,  # This flag is enabled
    'seed': '42',
}

args = get_args(model_params)

# Check
print(args.batch_size)
print(args.debug)
print(args.unseen_pheno)
print(args.seed)

8
True
True
[42]


## prepare file stats.csv (NO TOUCH!!)

In [ ]:
df_patient=pd.read_csv('/content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/03 Data/PATIENTS.csv')
df_admissions=pd.read_csv('/content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/03 Data/ADMISSIONS.csv')

In [ ]:
df_patient

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
46515,31840,44089,M,2026-05-25 00:00:00,NaN,NaN,NaN,0
46516,31841,44115,F,2124-07-27 00:00:00,NaN,NaN,NaN,0
46517,31842,44123,F,2049-11-26 00:00:00,2135-01-12 00:00:00,2135-01-12 00:00:00,NaN,1
46518,31843,44126,F,2076-07-25 00:00:00,NaN,NaN,NaN,0


In [ ]:
df_admissions

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1
58972,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1
58973,58596,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1
58974,58597,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0


In [ ]:
# cross-reference 2 datasets using subject_id and ROW_ID
df_patient_admissions = pd.merge(df_patient, df_admissions, on='SUBJECT_ID')

In [ ]:
#I remain the row_id of admission file
df_stats=df_patient_admissions[['SUBJECT_ID', 'HADM_ID', 'ROW_ID_y','GENDER','LANGUAGE','ETHNICITY']]

In [ ]:
df_stats.rename(columns={'ROW_ID_y': 'ROW_ID'}, inplace=True)

In [ ]:
df_stats.to_csv(os.path.join(data_dir, 'stats.csv'), index=False)

In [ ]:
df_stats=df_stats.set_index(['SUBJECT_ID', 'HADM_ID'])

In [ ]:
df_stats.drop(columns=['ROW_ID'], inplace=True)
df_stats.reset_index(inplace=True)

In [ ]:
df_stats.to_csv('stats.csv', index=False)

In [ ]:
key_to_check = (11216, 188044)
if key_to_check in df_stats.index:
    row = df_stats.loc[key_to_check]
    # Process the row
else:
    print(f"Key {key_to_check} not found in index.")

In [ ]:
phenos.loc[key_to_check]

,,row_id,phenotype_label,operator
subject_id,hadm_id,,,
11216,188044,9756,none,JTW


the file stats is stocked in the directory: data_dir (generated file folder)

## execute the model

In [ ]:
mean = lambda l: sum(l)/len(l) if len(l) > 0 else .0


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
all_losses = {'train': [], 'val': [], 'test': []}


def indicators_to_spans(labels, idx = None):
    def add_span(idx, c, start, end):
        span = (idx, c, start, end)
        spans.add(span)

    spans = set()
    if args.label_encoding == 'multiclass':
        num_tokens = len(labels)
        num_classes = args.num_labels // 2
        start = None
        cat = -1
        for t in range(num_tokens):
            prev_tag = labels[t-1] if t > 0 else args.num_labels -1
            cur_tag = labels[t]

            if start is not None and cur_tag == cat + 1:
                continue
            elif start is not None:
                add_span(idx, cat // 2, start, t - 1)
                start = None

            if start is None and (cur_tag in [2*x for x in range(num_classes)]
                              or (prev_tag == (args.num_labels - 1)
                                  and cur_tag != (args.num_labels - 1))):
                start = t
                cat = int(cur_tag) // 2 * 2
    else:
        num_tokens, num_classes = labels.shape
        if args.label_encoding == 'bo':
            num_classes //= 2
        elif args.label_encoding == 'boe':
            num_classes //= 3

        for c in range(num_classes):
            start = None
            for t in range(num_tokens):
                if args.label_encoding == 'bo':
                    if start and (labels[t, 2 * c] == 1 or labels[t, 2 * c + 1] == 0):
                        add_span(idx, c, start, t - 1)
                        start = None
                    elif start and labels[t, 2 * c + 1] == 1:
                        continue
                    if labels[t, 2 * c] == 1:
                        start = t
                elif args.label_encoding == 'boe':
                    if not start and labels[t, 3 * c] == 1:
                        start = t
                    elif start and labels[t, 3 * c + 2] == 1:
                        add_span(idx, c, start, t - 1)
                        start = None
                else:
                    if start and labels[t,c] == 1:
                        continue
                    elif start and labels[t,c] == 0:
                        add_span(idx, c, start, t - 1)
                        start = None
                    elif labels[t,c] == 1 and t == (num_tokens - 1):
                        span = (idx, c, -1, -1)
                        spans.add(span)
                    elif labels[t,c] == 1:
                        start = t
    return spans


def id_to_label(labels):
    new_labels = []
    for l in labels:
        if l == (args.num_labels - 1):
            new_l = 'O'
        elif l % 2 == 0:
            new_l = 'B-%d'% (l // 2)
        else:
            new_l = 'I-%d'% (l // 2)
        new_labels.append(new_l)
    return new_labels

def f1_score(ys, preds):
    tp = len(preds & ys)
    fn = len(ys) - tp
    fp = len(preds) - tp
    f1 = (2 * tp / (2 * tp + fp + fn)) * 100 if tp + fp + fn > 0 else 0
    return f1

def recall_score(ys, preds):
    tp = len(preds & ys)
    fn = len(ys) - tp
    recall = tp / (tp + fn)
    return recall

def calc_metrics_spans(ys, preds, span_ys = None):
    all_preds = []
    all_ys = []
    for i, (y, pred) in enumerate(zip(ys, preds)):
        pred_spans = indicators_to_spans(pred, idx = i)
        all_preds.append(pred_spans)
        if span_ys is None:
            y = y.squeeze()
            y_spans = indicators_to_spans(y, idx = i)
            all_ys.append(y_spans)

    all_preds = set().union(*all_preds)
    if span_ys is None:
        all_ys = set().union(*all_ys)
    else:
        all_ys = set(span_ys)
    f1 = f1_score(all_ys, all_preds)

    perclass = {}
    for c in range(args.num_decs):
        sub_ys = {x for x in all_ys if x[1] == c}
        sub_preds = {x for x in all_preds if x[1] == c}
        perclass[c] = f1_score(sub_ys, sub_preds)

    return f1, all_preds, all_ys, perclass

def save_losses(model, crit, train_dataloader, val_dataloader, test_dataloader):
    train_losses = evaluate(model, train_dataloader, crit, return_losses = True)
    all_losses['train'].append(train_losses)
    val_losses = evaluate(model, val_dataloader, crit, return_losses = True)
    all_losses['val'].append(val_losses)
    test_losses = evaluate(model, test_dataloader, crit, return_losses = True)
    all_losses['test'].append(test_losses)

def evaluate(model, dataloader, crit, return_losses = False, return_preds = False):
    model.eval()
    outs, ys = [], []
    lens = []
    token_masks = []
    for batch in tqdm(dataloader, desc='Evaluation'):
        x = batch['input_ids']
        y = batch['labels']
        mask = batch['mask']
        if args.task == 'seq':
            ids = batch['ids']

        with torch.no_grad():
            logits = model.generate(x, mask)

        outs.append(logits)
        lens.extend([x.shape[0] for x in logits])
        ys.append(y)

        if 'token_mask' in batch:
            token_masks.append(batch['token_mask'])

    if args.label_encoding == 'multiclass':
        outs_stack = torch.cat([x.view(-1, args.num_labels) for x in outs], 0)
        ys_stack = torch.cat([x.view(-1) for x in ys], 0).to(device)
        preds = [x.squeeze() for x in outs]

        if args.use_crf:
            padded_outs = torch.nn.utils.rnn.pad_sequence(preds, batch_first=True)
            outs_mask = ~(padded_outs[:,:,0] == 0)
            preds = crit.decode(padded_outs, mask=outs_mask)
            preds_stack = torch.tensor([x for pred in preds for x in pred]).to(device)
            padded_ys = torch.nn.utils.rnn.pad_sequence([x.squeeze() for x in ys], batch_first=True)
            loss = -1 * crit(padded_outs, padded_ys, mask=outs_mask, reduction='mean')

        else:
            preds =  [x.argmax(-1) for x in preds]
            preds_stack = outs_stack.argmax(-1)
            loss = crit(outs_stack, ys_stack)
    else:
        outs_stack = torch.cat(outs, 1)
        ys_stack = torch.cat(ys, 1).to(device)
        loss = crit(outs_stack, ys_stack)

    losses = []
    offset = 0
    if return_losses:
        for ln in lens:
            sub_losses = loss[offset:offset+ln]
            offset += ln
            losses.append(sub_losses.mean().item())
        return losses


    loss = loss.mean()

    y = torch.cat(ys, 1).squeeze()

    if len(token_masks) > 0:
        print('len(token_masks) > 0')
        print('token_masks',token_masks,'length:',len(token_masks),'\n')
        print('ys_stack',ys_stack,'length:',len(ys_stack),'\n')
        print('preds_stack',preds_stack,'length:',len(preds_stack),'\n')
        token_masks = torch.cat(token_masks, 1).squeeze().to(device)
        print('token_masks.sum()',token_masks.sum())
        print('((ys_stack == preds_stack).float() * token_masks).sum() / token_masks.sum()',((ys_stack == preds_stack).float() * token_masks).sum() / token_masks.sum())
        acc = ((ys_stack == preds_stack).float() * token_masks).sum() / token_masks.sum() * 100
    else:
        print('len(token_masks) <0')
        print('(ys_stack == preds_stack).float().mean()',(ys_stack == preds_stack).float().mean())
        acc = (ys_stack == preds_stack).float().mean() * 100

    if 'all_spans' in dataloader.dataset.data[0]:
        all_spans = [x['all_spans'] for x in dataloader.dataset.data]
        span_ys = [(i, s['label'], s['token_start'], s['token_end']) for i, spans in enumerate(all_spans) for s in spans[0]]
    else:
        span_ys = None
    f1, span_preds, span_ys, perclass = calc_metrics_spans(ys, preds, span_ys)
    if return_preds:
        return span_preds, span_ys
    metrics_out = {}
    metrics_out['f1'] = f1
    metrics_out['acc'] = acc
    model.train()

    # genders = dataloader.dataset.stats['gender']
    # for g in set(genders):
    #     ids = [i for i,x in enumerate(genders) if x==g]
    #     sub_ys = torch.cat([x for i,x in enumerate(ys) if i in ids], 1).squeeze().cpu()
    #     sub_preds = torch.cat([x for i,x in enumerate(preds) if i in ids]).cpu()
    #     sub_acc = (sub_ys == sub_preds).float().mean() * 100
    #     print(g, sub_acc)

    # ethnicities = dataloader.dataset.stats['ethnicity']
    # for e in set(ethnicities):
    #     ids = [i for i,x in enumerate(ethnicities) if x==e]
    #     sub_ys = torch.cat([x for i,x in enumerate(ys) if i in ids], 1).squeeze().cpu()
    #     sub_preds = torch.cat([x for i,x in enumerate(preds) if i in ids]).cpu()
    #     sub_acc = (sub_ys == sub_preds).float().mean() * 100
    #     print(e, sub_acc)

    # langs = dataloader.dataset.stats['language']
    # for l in set(langs):
    #     ids = [i for i,x in enumerate(langs) if x==l]
    #     sub_ys = torch.cat([x for i,x in enumerate(ys) if i in ids], 1).squeeze().cpu()
    #     sub_preds = torch.cat([x for i,x in enumerate(preds) if i in ids]).cpu()
    #     sub_acc = (sub_ys == sub_preds).float().mean() * 100
    #     print(l, sub_acc)

    if args.task == 'token':
        pheno_results = {}
        for pheno, ids in dataloader.dataset.pheno_ids.items():
            sub_ys = [x for i,x in enumerate(ys) if i in ids]
            sub_preds = [x for i,x in enumerate(preds) if i in ids]
            f1, span_preds, span_ys, _ = calc_metrics_spans(sub_ys, sub_preds)
            pheno_results[pheno] = f1
    else:
        pheno_results = None
    return metrics_out, pheno_results, loss, perclass

def process(sample, model, tokenizer, out_dir):
    hadm = sample['HADM_ID']
    fn = f"{sample['SUBJECT_ID']}_{int(hadm) if not pd.isnull(hadm) else 'NaN'}_{sample['ROW_ID']}.json"
    out_file = out_dir + fn
    if not os.path.exists(out_file):
        encoding = tokenizer.encode_plus(sample['TEXT'])
        x = torch.tensor(encoding['input_ids']).unsqueeze(0).to(device)
        mask = torch.tensor(encoding['attention_mask']).unsqueeze(0).to(device)

        with torch.no_grad():
            out = model.generate(x, mask)

        if args.label_encoding == 'multiclass':
            pred = out.argmax(-1)
        else:
            pred = torch.where(out > 0, 1, 0)
        pred = pred.squeeze()
        spans = indicators_to_spans(pred)
        all_spans = []
        for _, cat, start, end in spans:
            span_dict = {}
            span_dict['decision'] = sample['TEXT'][start:end]
            span_dict['category'] = 'Category %d'%(cat+1)
            span_dict['start_offset'] = start
            span_dict['end_offset'] = end
            all_spans.append(span_dict)
        with open(out_file, 'w') as f:
            json.dump(all_spans, f)

def predict_mimic(model, data, tokenizer):

    model.eval()
    outs = []
    out_dir = './all_mimic_decisions/'
    kwargs = {'model': model, 'tokenizer': tokenizer, 'out_dir': out_dir}
    import multiprocessing as mp
    from functools import partial
    mp.set_start_method('spawn', force=True)
    process_ = partial(process, **kwargs)
    pool = mp.Pool(3)
    for _ in tqdm(pool.imap_unordered(process_, data, chunksize=20000), total=len(data)):
        pass
    pool.close()
    # for sample in tqdm(data):
    #     process(sample)

def train(args, model, crit, optimizer, lr_scheduler,
        train_dataloader, val_dataloader, verbose=True, train_ns=None, test_dataloader=None):
    writer = aim.Run(experiment=args.aim_exp, repo=args.aim_repo,
            system_tracking_interval=0) if not args.debug else None
    if writer is not None:
        writer['hparams'] = args.__dict__

    step = 0
    best_f1 = -1
    best_acc = 0
    best_step = 0
    best_pheno = None
    best_perclass = None
    train_iter = iter(train_dataloader)
    losses = []
    while step < args.total_steps:
        batch = next(train_iter, None)
        if batch is None:
            train_iter = iter(train_dataloader)
            continue
        x = batch['input_ids']
        y = batch['labels']
        mask = batch['mask']

        y = y.to(device)
        if args.task == 'seq':
            out, _ = model.phenos(x, mask)
            logits = out[1]
        elif args.task == 'token':
            out, logits = model.decisions(x, mask)

        if args.label_encoding == 'multiclass':
            if args.use_crf:
                loss = -1 * crit(logits, y, reduction='mean')
            else:
                loss = crit(logits.view(-1, args.num_labels), y.view(-1)).mean()
        else:
            loss = crit(logits, y).mean()
        total_loss = loss


        losses.append(loss.item())
        total_loss /= args.grad_accumulation
        total_loss.backward(retain_graph=True)

        if (step+1) % args.grad_accumulation == 0:
            optimizer.step()
            optimizer.zero_grad()
            lr_scheduler.step()

        if step % (args.train_log*args.grad_accumulation) == 0:
            avg_loss = np.mean(losses)
            if verbose:
                print('step %d - training loss: %.3f'%(step, avg_loss))
            if writer is not None:
                writer.track(avg_loss, name='bce_loss', context={'split': 'train'}, step = step)
            losses = []

        if len(val_dataloader) > 0 and step % (args.val_log*args.grad_accumulation) == 0:
            if args.save_losses:
                save_losses(model, crit, train_ns, val_dataloader, test_dataloader)
            metrics_out, pheno_results, loss, perclass = evaluate(model, val_dataloader, crit)
            f1, acc = metrics_out['f1'], metrics_out['acc']
            if verbose:
                print('[step: {:5d}] f1: {:.1f}, acc: {:.1f}, loss: {:.3f}'
                        .format(step, f1, acc, loss))
            if writer is not None:
                writer.track(loss, name='bce_loss', context={'split': 'val'}, step = step)
                writer.track(f1, name='f1', step = step)
                # writer.track(prec, name='precision', step = step)
                # writer.track(rec, name='recall', step = step)
            if f1 > best_f1:
                best_f1 = f1
                best_acc = acc
                best_step = step
                best_pheno = pheno_results
                # best_perclass = metrics_out[4:6]
                if not args.debug:
                    torch.save(model.state_dict(), args.ckpt_dir)
        step += 1
    if writer is not None:
        writer.track(best_f1, name = 'best_f1')
        writer.track(best_step, name = 'best_step')
        if best_pheno is not None:
            for pheno, f1 in best_pheno.items():
                writer.track(f1, name='best_f1', context={'group': pheno})
        if args.task == 'token':
            f1s = best_perclass
            for i in range(len(f1s)):
                writer.track(f1s[i], name='best_f1', context={'decision': i})
    return best_f1, best_acc, best_step

def main(args):
    f1s = []
    for seed in args.seed:
        torch.manual_seed(seed)
        np.random.seed(seed)
        args.seed = seed
        train_dataloader, val_dataloader, test_dataloader, train_ns = load_data(args)
        model, crit, optimizer, lr_scheduler = load_model(args, device)

        if not args.eval_only:
            f1, acc, step = train(args, model, crit,
                    optimizer, lr_scheduler, train_dataloader,
                    val_dataloader, args.verbose, train_ns, test_dataloader)
            f1s.append(f1)
            print('seed: %d, F1: %.1f, Acc: %.1f'%(seed, f1, acc))
            # Test
            metrics_out, pheno_results, loss, perclass = evaluate(model, test_dataloader, crit)
            f1, acc = metrics_out['f1'], metrics_out['acc']
            print('[Test] f1: {:.1f}, acc: {:.1f}, loss: {:.3f}'
                    .format(f1, acc, loss))
            # print(pheno_results)
            print(perclass)
        else:
            model.eval()
            # Train
            # metrics_out, pheno_results, loss = evaluate(model, train_ns, crit)
            # f1, acc = metrics_out['f1'], metrics_out['acc']
            # print('[Train] f1: {:.1f}, acc: {:.1f}, loss: {:.3f}'
            #         .format(f1, acc, loss))

            # Val
            # metrics_out, pheno_results, loss, perclass = evaluate(model, val_dataloader, crit)
            # f1, acc = metrics_out['f1'], metrics_out['acc']
            # print('[Val] f1: {:.1f}, acc: {:.1f}, loss: {:.3f}'
            #         .format(f1, acc, loss))

            # Test
            metrics_out, pheno_results, loss, perclass = evaluate(model, test_dataloader, crit)
            f1, acc = metrics_out['f1'], metrics_out['acc']
            print('[Test] f1: {:.1f}, acc: {:.1f}, loss: {:.3f}'
                    .format(f1, acc, loss))
            # print(pheno_results)
            print(perclass)

            # predict_mimic(model, data, tokenizer)
        if args.save_losses:
            np.savez('losses_%d.npz'%seed, train=all_losses['train'], val=all_losses['val'], test=all_losses['test'])
    return np.mean(f1s)

if __name__ == '__main__':
    main(args)


Train dataset: 15
Val dataset: 3
Test dataset: 6
step 0 - training loss: 3.134


Evaluation: 100%|██████████| 3/3 [00:52<00:00, 17.61s/it]


len(token_masks) > 0
token_masks [tensor([[1., 1., 1.,  ..., 1., 1., 1.]], dtype=torch.float64), tensor([[1., 1., 1.,  ..., 1., 1., 1.]], dtype=torch.float64), tensor([[1., 1., 1.,  ..., 1., 1., 1.]], dtype=torch.float64)] length: 3 

ys_stack tensor([18, 18, 18,  ..., 18, 18, 18]) length: 10573 

preds_stack tensor([ 8,  3,  3,  ..., 14, 14,  8]) length: 10573 

token_masks.sum() tensor(10573., dtype=torch.float64)
((ys_stack == preds_stack).float() * token_masks).sum() / token_masks.sum() tensor(0.0232, dtype=torch.float64)
[step:     0] f1: 0.0, acc: 2.3, loss: 3.134


KeyboardInterrupt: 